# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [572]:
# url = https://jportal.mdcourts.gov/license/index_disclaimer.jsp

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages.

### How will you identify the checkbox to check it?

Selenium can find elements by:

- name
- Class
- ID
- CSS selector (**ASK ME WHAT THIS IS** if you don't know)
- XPath (**ASK ME WHAT THIS IS** because you definitely don't know)
- Link text
- Partial link text

So in other words, what's unique about this element?

- *TIP: I have a secret awesome way to do this, but you have to ask.*

### How will you identify the button to select it, or the form to submit it?

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

In [573]:
from selenium import webdriver
driver = webdriver.Chrome()
driver.get("https://jportal.mdcourts.gov/license/index_disclaimer.jsp")

In [574]:
check_box = driver.find_element_by_xpath('//*[@id="checkbox"]')
check_box.click()

In [575]:
from selenium.webdriver.common.keys import Keys

In [576]:
element = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
element.send_keys(Keys.RETURN)

In [577]:
a = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[6]/td[2]/a[2]')
a.send_keys(Keys.RETURN)

### Once you're on the next page, how will you click the "SEARCH LICENSE RECORDS" link or follow its URL?

In [578]:
from selenium.webdriver.support.ui import Select

### Now you're on the form page. How do you pick "Statewide" for the jurisdiction dropdown?

In [579]:
select = Select(driver.find_element_by_xpath('//*[@id="slcJurisdiction"]'))
#Select = Select(select_tag)
select.select_by_visible_text('Statewide')

### How do you type "vap%" into the Trade Name field?

In [580]:
trade_input = driver.find_element_by_xpath('//*[@id="txtTradeName"]')
trade_input.send_keys('vap%')

### How do you click the submit button or submit the form?

In [581]:
submit = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]')
submit.click()

### How can you find and click the 'Next' button on the search results page?

# Okay, let's scrape!

### Use Selenium to search for vape shops statewide, and then click through until it reaches the last page.

Don't scrape yet!

### Use Selenium to scrape the first page of search results for vape shops statewide (well, we'll try).

- *TIP: You can find elements/text using Selenium, or use BeautifulSoup with `doc = BeautifulSoup(driver.page_source)`*

I've included some BeautifulSoup code that might be helpful. If you use it, **ask me how I made it.** It's important.

In [582]:
from bs4 import BeautifulSoup
import requests

In [583]:
# IF YOU ARE USING BEAUTIFULSOUP, HERE IS SOME SAMPLE CODE
# IF YOU USE THIS CODE, ASK ME HOW I MADE IT. IT'S IMPORTANT.


In [584]:
import re

In [585]:
# You'll probably need to find specific tds inside
# of the rows instead of looking at the whole thing.
vap_list = []

        

while True:
    doc = BeautifulSoup(driver.page_source, 'html.parser')
    business_headers = doc.find_all('tr',class_='searchfieldtitle')
    len(business_headers)
    
    for header in business_headers:
        vap_dict = {}
        rows = header.find_next_siblings('tr')
        
        name = header.find('span', attrs = {'class': 'copybold'}).text.strip()
        vap_dict["name"] = name
        
        web = header.find_all('td')[2]
        if web.find('a'):
            vap_dict['link'] = web.a['href']
        else:
            vap_dict['link'] = web.text.strip()
        
        vap_dict["owner's name"] = rows[0].find_all('td')[1].text.strip()
        vap_dict["street"]= rows[1].find_all('td')[1].text.strip()
        vap_dict["county"]= rows[3].text.strip()
        vap_dict["city"]= rows[2].find_all('td')[1].text.strip()

        # split
        date = rows[2].find_all('td')[2].text.strip()
        date_d = date.split(':')
        if len(date_d) > 1:
            vap_dict["issue date"] = date_d[1].strip()


        lic = rows[0].find_all('td')[2].text.strip()
        lic_s = lic.split(':')
        if len(lic_s) > 1:
            vap_dict["lic. Status"]= lic_s[1].strip()

        lic_n = rows[1].find_all('td')[2].text.strip()
        number = lic_n.split(':')
        if len(number) > 1:
            vap_dict["license"]= number[1].strip()

        vap_list.append(vap_dict)
    try:   
        # Hey, to get that link and click it
        next_button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a/nobr')
        next_button.click()
    
    except:
    # Oh did you get an error? Exit the while loop
        break

vap_list

[{'city': 'SALISBURY, MD 21801',
  'county': 'Wicomico County',
  'issue date': '4/27/2017',
  'lic. Status': 'Issued',
  'license': '22173807',
  'link': 'pbLicenseDetail.jsp?owi=ER9BOw3Fc48%3D',
  'name': 'VAPE IT STORE I',
  "owner's name": 'AMIN NARGIS',
  'street': '1724 N SALISBURY BLVD UNIT 2'},
 {'city': 'SALISBURY, MD 21801',
  'county': 'Wicomico County',
  'issue date': '4/27/2017',
  'lic. Status': 'Issued',
  'license': '22173808',
  'link': 'pbLicenseDetail.jsp?owi=eDt7Wd3p1WM%3D',
  'name': 'VAPE IT STORE II',
  "owner's name": 'AMIN NARGIS',
  'street': '1015 S SALISBURY BLVD'},
 {'city': 'GAMBRILLS, MD 21054',
  'county': 'Anne Arundel County',
  'issue date': '4/05/2017',
  'lic. Status': 'Issued',
  'license': '02104436',
  'link': 'pbLicenseDetail.jsp?owi=zeXxbPAhES8%3D',
  'name': 'VAPEPAD THE',
  "owner's name": 'ANJ DISTRIBUTIONS LLC',
  'street': '2299 JOHNS HOPKINS ROAD'},
 {'city': 'CHESTER, MD 21619',
  'county': "Queen Anne's County",
  'issue date': '5/31/2

### Save these into `vape-results.csv`

In [586]:
import pandas as pd
df = pd.DataFrame(vap_list)

### Open `vape-results.csv` to make sure there aren't any extra weird columns

In [587]:
df.head(39)

,city,county,issue date,lic. Status,license,link,name,owner's name,street
0,"SALISBURY, MD 21801",Wicomico County,4/27/2017,Issued,22173807,pbLicenseDetail.jsp?owi=ER9BOw3Fc48%3D,VAPE IT STORE I,AMIN NARGIS,1724 N SALISBURY BLVD UNIT 2
1,"SALISBURY, MD 21801",Wicomico County,4/27/2017,Issued,22173808,pbLicenseDetail.jsp?owi=eDt7Wd3p1WM%3D,VAPE IT STORE II,AMIN NARGIS,1015 S SALISBURY BLVD
2,"GAMBRILLS, MD 21054",Anne Arundel County,4/05/2017,Issued,02104436,pbLicenseDetail.jsp?owi=zeXxbPAhES8%3D,VAPEPAD THE,ANJ DISTRIBUTIONS LLC,2299 JOHNS HOPKINS ROAD
3,"CHESTER, MD 21619",Queen Anne's County,5/31/2017,Issued,17165957,pbLicenseDetail.jsp?owi=UOUT2JAKKSI%3D,VAPE FROG,COX TRADING COMPANY L L C,110 S. PINEY RD
4,"SEVERNA PARK, MD 21146",Anne Arundel County,NaN,Pending,NaN,Pending *,VAPE FROG,COX TRADING LLC,346 RITCHIE HIGHWAY
5,"EDGEWATER, MD 21037",Anne Arundel County,4/13/2017,Issued,02102408,pbLicenseDetail.jsp?owi=oQVbMddhTEU%3D,VAPE LOFT (THE),DISBROW II EMERSON HARRINGTON,185 MITCHELLS CHANCE RD
6,"COLUMBIA, MD 21045",Howard County,5/19/2017,Issued,13141786,pbLicenseDetail.jsp?owi=HmIiRb7lvqg%3D,VAPE N CIGAR,DISCOUNT TOBACCO ESSEX LLC,7104 MINSTREL UNIT #7
7,"WESTMINSTER, MD 21157",Carroll County,4/21/2017,Issued,06126253,pbLicenseDetail.jsp?owi=CAMU4qjs8jQ%3D,VAPE DOJO,FAIRGROUND VILLAGE LLC,330 ONE FORTY VILLAGE ROAD
8,"CHARLOTTE HALL, MD 20622",St. Mary's County,NaN,Pending,NaN,Pending *,VAPE HAVEN,GRIMM JENNIFER,29890 THREE NOTCH ROAD
9,"STEVENSVILLE, MD 21666",Queen Anne's County,4/13/2017,Issued,17166688,pbLicenseDetail.jsp?owi=H5xkBiif%2B3c%3D,VAPE BIRD,HUTCH VAPES LLC,356 ROMANCOKE ROAD


## Use Selenium to scrape ALL pages of results, save the results into `vape-results-all.csv`.

In [589]:
df.to_csv('vape-results-all.csv', index = False)

In [588]:
driver.close()